# Import relavant modules

In [16]:
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re
import warnings
warnings.filterwarnings("ignore")
import datamapper

In [2]:
pd.set_option('display.max_colwidth', -1)
pd.options.display.max_rows = 999

In [3]:
data = pd.read_csv('dataset.csv')

### Basic examination of data as df after a manual look

In [4]:
data.columns

Index(['id', 'name', 'tenant_id', 'address_line', 'city', 'state', 'zip',
       'country', 'phone', 'web', 'google_url', 'facebook_url', 'cars_url'],
      dtype='object')

In [651]:
data.head(5)

,id,name,tenant_id,address_line,city,state,zip,country,phone,web,google_url,facebook_url,cars_url
0,3244033,# 1 Brookville Chevrolet,130469,1 E Main St,Brookville,PA,15825,US,(814) 849-8313,https://www.brookvillegmdealer.com,https://maps.google.com/maps?cid=12622059353105481409,https://www.facebook.com/brookvillechevy/,https://www.cars.com/dealers/154863/1-brookville-chevrolet-buick/
1,18311,#1 BROOKVILLE CHEVROLET-BUICK,1840,ONE E MAIN ST,BROOKVILLE,PA,15825,US,(814) 849-8313,NaN,NaN,https://www.facebook.com/brookvillechevy/,NaN
2,118193,#1 BROOKVILLE CHEVROLET-BUICK,2980,1 East Main St,Brookville,PA,15825,US,(814) 849-8313,NaN,https://maps.google.com/maps?cid=12622059353105481409,https://www.facebook.com/brookvillechevy/,http://www.cars.com/dealers/154863/1-brookville-chevrolet-buick/
3,18248,#1 BROOKVILLE CHEVROLET-BUICK,1836,ONE E MAIN ST,BROOKVILLE,PA,15825,US,(814) 849-8313,NaN,NaN,NaN,NaN
4,575489,#1 BROOKVILLE CHEVROLET-BUICK,206,30 Main St,Brookville,PA,15825,US,(814) 849-8313,http://www.brookvillegmdealer.com,https://maps.google.com/maps?cid=12622059353105481409,https://www.facebook.com/brookvillechevy/,https://www.cars.com/dealers/154863/1-brookville-chevrolet-buick/


In [652]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            30000 non-null  int64 
 1   name          30000 non-null  object
 2   tenant_id     30000 non-null  int64 
 3   address_line  29998 non-null  object
 4   city          30000 non-null  object
 5   state         30000 non-null  object
 6   zip           29999 non-null  object
 7   country       30000 non-null  object
 8   phone         28117 non-null  object
 9   web           17358 non-null  object
 10  google_url    26356 non-null  object
 11  facebook_url  19583 non-null  object
 12  cars_url      18962 non-null  object
dtypes: int64(2), object(11)
memory usage: 3.0+ MB


In [653]:
data.describe()

,id,tenant_id
count,3.000000e+04,30000.000000
mean,1.663171e+06,58770.750133
std,1.119972e+06,50150.668922
min,7.937000e+03,10.000000
25%,5.751895e+05,4634.000000
50%,1.521736e+06,45332.000000
75%,2.777045e+06,115099.000000
max,3.454223e+06,139263.000000


In [654]:
data.shape

(30000, 13)

## Data cleaning
#### Standardize alphabet casing.
#### handle spaces and nulls for uniformity.
#### Convert single numbers from word to number.
#### Remove columns which is a same value through out DF.
#### Remove special charecters.
#### Handling nulls at important places.

In [656]:
data.columns

Index(['id', 'name', 'tenant_id', 'address_line', 'city', 'state', 'zip',
       'country', 'phone', 'web', 'google_url', 'facebook_url', 'cars_url'],
      dtype='object')

## Handling Non numeric columns

In [657]:
number_map = {
    'one': '1',
    'two': '2',
    'three': '3',
    'four': '4',
    'five': '5',
    'six': '6',
    'seven': '7',
    'eight': '8',
    'nine': '9',
    'zero' : '0'
}
number_map = {r"\b{}\b".format(k): v for k, v in number_map.items()}

In [658]:
non_numeric_columns = data.dtypes[data.dtypes != 'int64'][data.dtypes != 'float64'].index
non_numeric_columns

Index(['name', 'address_line', 'city', 'state', 'zip', 'country', 'phone',
       'web', 'google_url', 'facebook_url', 'cars_url'],
      dtype='object')

In [659]:
data[['name','address_line','zip','city','state','country']]=data[['name','address_line','zip','city','state','country']].replace('[^A-Za-z0-9]+',' ',regex=True)

for column in non_numeric_columns:
    data[column] = data[column].str.strip().str.lower()
    data[column] = data[column].replace(number_map, regex=True)
    data[column] = data[column].replace(r'^\s*$', np.NaN, regex=True)

    if (data[column].nunique() == 1) :
        data.drop([column], axis = 1, inplace = True)



Clearly as the Data is of dealers in U.S, the country column can be discarded

### Examining zip code uniformity

In [660]:
data[data['zip'].str.len().gt(5)]

## Incorrect zip, filling null

,id,name,tenant_id,address_line,city,state,zip,phone,web,google_url,facebook_url,cars_url
5,1575061,1 brookville chevrolet buick,54616,30 main st,brookville,pa,15825 1642,(814) 246-2055,http://www.brookvillegmdealer.com,https://maps.google.com/maps?cid=12622059353105481409,https://www.facebook.com/brookvillechevy/,https://www.cars.com/dealers/154863/1-brookville-chevrolet-buick/
7,1271984,1 brookville chevrolet buick,23144,30 main st,brookville,pa,15825 1642,(877) 223-2395,http://www.brookvillegmdealer.com,https://maps.google.com/maps?cid=12622059353105481409,https://www.facebook.com/brookvillechevy/,https://www.cars.com/dealers/154863/1-brookville-chevrolet-buick/
10,114365,1 cochran mazda,1057,4515 william penn hwy,monroeville,pa,15146 2813,(412) 373-3333,NaN,https://maps.google.com/maps?cid=12743786671890848048,https://www.facebook.com/numberonecochran/,https://www.cars.com/dealers/5247830/cochran-mazda-monroeville/
25,1273660,1 cochran buick gmc of monroeville,23144,4520 william penn hwy,monroeville,pa,15146 2814,(877) 304-9607,http://www.cochranofmonroeville.com,https://maps.google.com/maps?cid=2752462366417878165,NaN,NaN
40,1273111,1 cochran cadillac of monroeville,23144,4520 william penn hwy,monroeville,pa,15146 2814,(877) 824-2498,http://www.cochrancadillac.com,https://maps.google.com/maps?cid=17115351350047781923,NaN,https://www.cars.com/dealers/202604/1-cochran-buick-gmc-monroeville/
...,...,...,...,...,...,...,...,...,...,...,...,...
29922,1146012,buena park nissan,2552,6501 auto center dr,buena park,ca,90621 2901,(714) 739-0800,NaN,https://maps.google.com/maps?cid=12617323471123278991,https://www.facebook.com/buenaparknissan/,https://www.cars.com/dealers/106379/buena-park-nissan/
29945,1937104,buerkle honda,10726,3360 highway 61 n,saint paul,mn,55110 5212,(651) 490-6600,https://www.buerklehonda.com/,https://maps.google.com/maps?cid=885141847253462911,https://www.facebook.com/buerklehondawbl/,https://www.cars.com/dealers/85763/buerkle-automotive/
29958,1275236,buff whelan chevrolet,23144,40445 van dyke ave,sterling heights,mi,48313 3736,(586) 933-5557,http://www.buffwhelan.com,https://maps.google.com/maps?cid=8836726731726321501,https://www.facebook.com/nickcarriercarsales/,https://www.cars.com/dealers/1405/buff-whelan-chevrolet/
29961,94689,buffalo,3848,6755 transit rd,east amherst,ny,14051 1485,(716) 636-6000,NaN,https://maps.google.com/maps?cid=9746768870518867160,https://www.facebook.com/195215093949157,http://www.cars.com/dealers/176319/enterprise-car-sales-buffalo/


In [661]:
## Lets derive a short zip from long zips
data['zip_short']  = data.zip.apply(lambda x: str(x).split(' ')[0] if len(str(x))>5 else x)

In [662]:
data.zip = data.zip.apply(lambda x: x if str(x).replace(' ','').isdigit() else np.nan)
data.zip_short = data.zip_short.apply(lambda x: x if str(x).replace(' ','').isdigit() else np.nan)

In [663]:
data.zip.isnull().sum()

2

In [664]:
data.zip_short.isnull().sum()

2

Seems to be a bad row, but the zip and address line can be extracted using Geocoding API from google maps url, which is too advanced for now, so removing it instead

In [665]:
data = data[~data.zip.isnull()]

In [666]:
data.isnull().sum()

id              0    
name            2    
tenant_id       0    
address_line    1    
city            0    
state           0    
zip             0    
phone           1883 
web             12642
google_url      3644 
facebook_url    10415
cars_url        11036
zip_short       0    
dtype: int64

# Generalize web addresses only upto .com

In [667]:
data.web.head(10)

0    https://www.brookvillegmdealer.com
1    NaN                               
2    NaN                               
3    NaN                               
4    http://www.brookvillegmdealer.com 
5    http://www.brookvillegmdealer.com 
6    NaN                               
7    http://www.brookvillegmdealer.com 
8    https://www.brookvillegmdealer.com
9    http://www.ricartkia.com/         
Name: web, dtype: object

In [668]:
data.web = data.web.str.strip().str.replace('https://','').str.replace('http://','')
data.web

0        www.brookvillegmdealer.com
1        NaN                       
2        NaN                       
3        NaN                       
4        www.brookvillegmdealer.com
                    ...            
29995    NaN                       
29996    www.buickgmcfalls.com/    
29997    NaN                       
29998    www.buickgmcfalls.com     
29999    NaN                       
Name: web, Length: 29998, dtype: object

In [669]:
data.web = data.web.apply(lambda x: str(x).split('/')[0] if x is not None else np.nan)
data.web = data.web.replace('nan',np.nan)

In [670]:
data.web.head(10)

0    www.brookvillegmdealer.com
1    NaN                       
2    NaN                       
3    NaN                       
4    www.brookvillegmdealer.com
5    www.brookvillegmdealer.com
6    NaN                       
7    www.brookvillegmdealer.com
8    www.brookvillegmdealer.com
9    www.ricartkia.com         
Name: web, dtype: object

In [671]:
data.web.isnull().sum()

12642

## Data with no dealer name available

In [672]:
data[data.name.isnull()] 

# These rows are useless as well. removing them.

,id,name,tenant_id,address_line,city,state,zip,phone,web,google_url,facebook_url,cars_url,zip_short
160,2728616,NaN,109662,755 i 45,huntsville,tx,77340,NaN,NaN,NaN,NaN,NaN,77340
161,2733888,NaN,114950,755 i 45,huntsville,tx,77340,NaN,NaN,https://maps.google.com/maps?cid=7613114450716243350,NaN,NaN,77340


In [673]:
data = data[~data.name.isnull()]

In [674]:
data.isnull().sum()

id              0    
name            0    
tenant_id       0    
address_line    1    
city            0    
state           0    
zip             0    
phone           1881 
web             12640
google_url      3643 
facebook_url    10413
cars_url        11034
zip_short       0    
dtype: int64

In [675]:
data[data.address_line.isnull()]

# Except addressline, everything is present, lets retain this for further uses

,id,name,tenant_id,address_line,city,state,zip,phone,web,google_url,facebook_url,cars_url,zip_short
1707,2515038,acura turnersville,109662,NaN,washington township,nj,8012,(856) 497-4749,www.acuraturnersville.com,https://maps.google.com/maps?cid=2201156893789197068,NaN,NaN,8012


In [676]:
data.shape

(29996, 13)

### Similarly phone number can be standardized if there are any issues. Leaving that for now

# Inspect no of unique entries per columns

In [677]:
for column in data.columns:
    print("The " + column  + " has following unique elements :" + str(data[column].nunique()) + " out of " + str(data.shape[0]) + " entries")

The id has following unique elements :29664 out of 29996 entries
The name has following unique elements :7970 out of 29996 entries
The tenant_id has following unique elements :661 out of 29996 entries
The address_line has following unique elements :7916 out of 29996 entries
The city has following unique elements :1882 out of 29996 entries
The state has following unique elements :53 out of 29996 entries
The zip has following unique elements :4264 out of 29996 entries
The phone has following unique elements :7657 out of 29996 entries
The web has following unique elements :4770 out of 29996 entries
The google_url has following unique elements :5546 out of 29996 entries
The facebook_url has following unique elements :7878 out of 29996 entries
The cars_url has following unique elements :4986 out of 29996 entries
The zip_short has following unique elements :2750 out of 29996 entries


### As first step remove duplicates compared in whole data

In [678]:
duplicate_whole = data[data.duplicated(data.columns)]

In [679]:
duplicate_whole.shape

(32, 13)

In [680]:
data = data.drop_duplicates(subset = data.columns, keep = 'first')
data.shape

(29964, 13)

In [681]:
data.isnull().sum()

id              0    
name            0    
tenant_id       0    
address_line    1    
city            0    
state           0    
zip             0    
phone           1881 
web             12640
google_url      3643 
facebook_url    10413
cars_url        11034
zip_short       0    
dtype: int64

### There seems to be that there are few duplicates based on ID, lets inspect them if they are pure duplicates or is there any info we can fetch from them?

In [682]:
duplicate = data[data['id'].duplicated(keep=False)]
duplicate.shape

(509, 13)

###  Upon manual inspection in file it can be seen these are pure duplicates except for changes in URLs, lets double check using the other metadata except URLs

In [683]:
duplicate2 = data[data.duplicated(subset = ['id','name','tenant_id','address_line','city','state','zip','phone'], keep=False)]
duplicate2.shape

(509, 13)

Our assumption turned out to be true, there are 509 duplicates where the only variations lie at URLs like web google_url etc. As we dont exactly know which URL is correct, lets pick one value among them and lets merge tenants.

In [684]:
duplicate2.isnull().sum()

id              0  
name            0  
tenant_id       0  
address_line    0  
city            0  
state           0  
zip             0  
phone           24 
web             269
google_url      22 
facebook_url    115
cars_url        53 
zip_short       0  
dtype: int64

## Try to fill information from similar rows at nulls, which seem to had no impact over nulls.

# Go ahead and remove duplicates based on metadata and retain only 1 value

In [685]:
data = data.drop_duplicates(subset = ['id','name','tenant_id','address_line','city','state','zip','phone'], keep = 'first')
data.shape

(29664, 13)

We have a reduce of over 350 entries.

Lets check number of dealers based on name

In [686]:
data.isnull().sum()

id              0    
name            0    
tenant_id       0    
address_line    1    
city            0    
state           0    
zip             0    
phone           1869 
web             12495
google_url      3632 
facebook_url    10352
cars_url        11004
zip_short       0    
dtype: int64

#### inspection and consolidation based on name

In [687]:
data.name.value_counts()

bruckner s mack volvo                     35
affordable auto sales                     29
affordable autos                          28
anderson ford                             27
all american chevrolet                    25
                                          ..
bayway cadillac of the woodlands 17010    1 
b j maurer motor company 07428            1 
bosak motor sales                         1 
bergstrom enterprise motorcars            1 
audi of roseville                         1 
Name: name, Length: 7970, dtype: int64

In [688]:
data[data.name == "bruckner s mack volvo"]

,id,name,tenant_id,address_line,city,state,zip,phone,web,google_url,facebook_url,cars_url,zip_short
29402,1605023,bruckner s mack volvo,86817,1125 slaton rd,lubbock,tx,79404,(888) 999-0491,www.brucknertruck.com,https://maps.google.com/maps?cid=13516605791043902826,NaN,NaN,79404
29403,1594486,bruckner s mack volvo,57705,8101 daytona rd nw,albuquerque,nm,87121,(877) 999-2451,www.brucknertruck.com,https://maps.google.com/maps?cid=7718685067242739490,NaN,NaN,87121
29404,2739739,bruckner s mack volvo,114950,4050 bloomfield hwy,farmington,nm,87401,(877) 999-2971,NaN,https://maps.google.com/maps?cid=9380787747266475927,NaN,NaN,87401
29405,1507833,bruckner s mack volvo,45332,4275 meadow ln,bossier city,la,71111,(866) 928-6580,www.brucknertruck.com,https://maps.google.com/maps?cid=11146096563401691841,NaN,NaN,71111
29406,1594623,bruckner s mack volvo,57705,4050 bloomfield hwy,farmington,nm,87401,(877) 999-2971,www.brucknertruck.com,https://maps.google.com/maps?cid=9380787747266475927,https://www.facebook.com/brucknertruckfarmingtonnm/,NaN,87401
29407,2743175,bruckner s mack volvo,115099,2509 n taylor ave,garden city,ks,67846,(888) 999-7810,NaN,https://maps.google.com/maps?cid=10582764957551610916,NaN,NaN,67846
29408,1512170,bruckner s mack volvo,45332,2121 nw county rd,hobbs,nm,88240,(877) 999-1440,www.brucknertruck.com,https://maps.google.com/maps?cid=11187758810578127375,https://www.facebook.com/pages/bruckners-truck-sales/167522453260378,NaN,88240
29409,1518181,bruckner s mack volvo,45332,9471 i 40 east exit 76,amarillo,tx,79118,(877) 999-8071,www.brucknertruck.com,https://maps.google.com/maps?cid=5404558478043464866,https://www.facebook.com/brucknertrucksales/,NaN,79118
29410,1603756,bruckner s mack volvo,86817,2509 n taylor ave,garden city,ks,67846,(888) 999-7810,www.brucknertruck.com,https://maps.google.com/maps?cid=10582764957551610916,NaN,NaN,67846
29411,2735232,bruckner s mack volvo,114950,2121 nw county rd,hobbs,nm,88240,(877) 999-1440,NaN,https://maps.google.com/maps?cid=11187758810578127375,NaN,NaN,88240


# There seem to be having entries with same name, address, city and state but different IDs, such entries can be consolidated.

In [689]:
data['meta_data'] = data['name'].fillna(' ') + ',' + data['address_line'].fillna(' ') + ',' + data['city'].fillna(' ')\
+data['state'].fillna(' ') + ',' + data['zip_short'].fillna(' ')

In [690]:
data['meta_data']

0        1 brookville chevrolet,1 e main st,brookvillepa,15825                  
1        1 brookville chevrolet buick,1 e main st,brookvillepa,15825            
2        1 brookville chevrolet buick,1 east main st,brookvillepa,15825         
3        1 brookville chevrolet buick,1 e main st,brookvillepa,15825            
4        1 brookville chevrolet buick,30 main st,brookvillepa,15825             
                                    ...                                         
29995    buick gmc menomonee falls,n88 w14167 main st,menomonee fallswi,53051   
29996    buick gmc menomonee falls,n88 w14167 main st,menomonee fallswi,53051   
29997    buick gmc menomonee falls,main st,menomonee fallswi,53051              
29998    buick gmc menomonee falls,n88 w14167 main st,menomonee fallswi,53051   
29999    buick gmc menomonee falls,n88 w14167 main st,n88 w14167 main stwi,53051
Name: meta_data, Length: 29664, dtype: object

lets also map numbers from 1 to 9 with corresponding value

In [691]:
data["meta_data"].nunique()

13523

### Out of 29664 entries only 13523 entries are unique which means around 50% of data is repeated based on only name,address,city and state. Lets work on them

In [692]:
duplicated_metadata =  data[data.duplicated(subset = ['meta_data'], keep=False)]

In [693]:
duplicated_metadata.shape

(21920, 14)

In [694]:
duplicated_metadata

,id,name,tenant_id,address_line,city,state,zip,phone,web,google_url,facebook_url,cars_url,zip_short,meta_data
1,18311,1 brookville chevrolet buick,1840,1 e main st,brookville,pa,15825,(814) 849-8313,NaN,NaN,https://www.facebook.com/brookvillechevy/,NaN,15825,"1 brookville chevrolet buick,1 e main st,brookvillepa,15825"
2,118193,1 brookville chevrolet buick,2980,1 east main st,brookville,pa,15825,(814) 849-8313,NaN,https://maps.google.com/maps?cid=12622059353105481409,https://www.facebook.com/brookvillechevy/,http://www.cars.com/dealers/154863/1-brookville-chevrolet-buick/,15825,"1 brookville chevrolet buick,1 east main st,brookvillepa,15825"
3,18248,1 brookville chevrolet buick,1836,1 e main st,brookville,pa,15825,(814) 849-8313,NaN,NaN,NaN,NaN,15825,"1 brookville chevrolet buick,1 e main st,brookvillepa,15825"
4,575489,1 brookville chevrolet buick,206,30 main st,brookville,pa,15825,(814) 849-8313,www.brookvillegmdealer.com,https://maps.google.com/maps?cid=12622059353105481409,https://www.facebook.com/brookvillechevy/,https://www.cars.com/dealers/154863/1-brookville-chevrolet-buick/,15825,"1 brookville chevrolet buick,30 main st,brookvillepa,15825"
5,1575061,1 brookville chevrolet buick,54616,30 main st,brookville,pa,15825 1642,(814) 246-2055,www.brookvillegmdealer.com,https://maps.google.com/maps?cid=12622059353105481409,https://www.facebook.com/brookvillechevy/,https://www.cars.com/dealers/154863/1-brookville-chevrolet-buick/,15825,"1 brookville chevrolet buick,30 main st,brookvillepa,15825"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29992,1519947,buick gmc fort walton beach,45332,329 miracle strip pkwy sw,fort walton beach,fl,32548,(850) 244-5165,www.buickgmcfortwaltonbeach.com,https://maps.google.com/maps?cid=1201605996612827940,https://www.facebook.com/buickcadillacgmcftwalton,https://www.cars.com/dealers/5380513/buick-gmc-cadillac-fort-walton-beach/,32548,"buick gmc fort walton beach,329 miracle strip pkwy sw,fort walton beachfl,32548"
29993,3250701,buick gmc fort walton beach,130469,329 miracle strip pkwy sw,fort walton beach,fl,32548,(850) 364-1032,www.buickgmcfortwaltonbeach.com,https://maps.google.com/maps?cid=4601099509292193902,https://www.facebook.com/buickcadillacgmcftwalton/,NaN,32548,"buick gmc fort walton beach,329 miracle strip pkwy sw,fort walton beachfl,32548"
29995,2747944,buick gmc menomonee falls,115107,n88 w14167 main st,menomonee falls,wi,53051,(262) 255-6000,NaN,https://maps.google.com/maps?cid=10136321579153410390,https://www.facebook.com/buick-gmc-menomonee-falls-276258956513411,NaN,53051,"buick gmc menomonee falls,n88 w14167 main st,menomonee fallswi,53051"
29996,1596983,buick gmc menomonee falls,57705,n88 w14167 main st,menomonee falls,wi,53051,(262) 255-6000,www.buickgmcfalls.com,https://maps.google.com/maps?cid=10136321579153410390,https://www.facebook.com/ernievonschledornbuickgmc/,https://www.cars.com/dealers/16763/ernie-von-schledorn-buick-gmc-volkswagen/,53051,"buick gmc menomonee falls,n88 w14167 main st,menomonee fallswi,53051"


In [695]:
data_combined=data.groupby(['meta_data']).first().reset_index()

In [696]:
data_combined.shape

(13523, 14)

In [697]:
data_combined

,meta_data,id,name,tenant_id,address_line,city,state,zip,phone,web,google_url,facebook_url,cars_url,zip_short
0,"001 putnam chevrolet and cadillac,3052 martin luther king junior blvd,burlingameca,94031",1123420,001 putnam chevrolet and cadillac,11138,3052 martin luther king junior blvd,burlingame,ca,94031,(479) 856-6390,www.bigotires.com,NaN,https://www.facebook.com/bigotiresfayetteville/,NaN,94031
1,"002 fremont chevrolet,5850 cushing pkwy,fremontca,94538",1123421,002 fremont chevrolet,11138,5850 cushing pkwy,fremont,ca,94538,(928) 344-2702,NaN,https://maps.google.com/maps?cid=2529089401223706257,https://www.facebook.com/bigotiresyuma004006/,NaN,94538
2,"003 chevrolet of stevens creek,3640 stevens creek blvd,san joseca,95117",1123422,003 chevrolet of stevens creek,11138,3640 stevens creek blvd,san jose,ca,95117,(520) 625-9414,NaN,https://maps.google.com/maps?cid=17294707326087392923,https://www.facebook.com/bigotiresgreenvalley004018/,NaN,95117
3,"004 capitol chevrolet,905 capitol expressway auto mall,san joseca,95136",1123423,004 capitol chevrolet,11138,905 capitol expressway auto mall,san jose,ca,95136,(928) 776-1111,NaN,https://maps.google.com/maps?cid=16621265989335969999,https://www.facebook.com/pages/big-o-tires/142282719150678,NaN,95136
4,"1 brookville chevrolet buick,1 e main st,brookvillepa,15825",18311,1 brookville chevrolet buick,1840,1 e main st,brookville,pa,15825,(814) 849-8313,NaN,NaN,https://www.facebook.com/brookvillechevy/,NaN,15825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13518,"byers kia oh057,5342 columbus pike,lewis centeroh,43035",16039,byers kia oh057,1157,5342 columbus pike,lewis center,oh,43035,NaN,www.byerskia.com,https://maps.google.com/maps?cid=16395390017796853957,NaN,http://www.cars.com/dealers/187255/byers-kia/,43035
13519,"byers mazda,2455 billingsley rd,columbusoh,43235",208328,byers mazda,1057,2455 billingsley rd,columbus,oh,43235 1925,(614) 792-2455,NaN,https://maps.google.com/maps?cid=1129683514267478914,https://www.facebook.com/byersmazdadublin/,https://www.cars.com/dealers/2575096/byers-dublin-mazda/,43235
13520,"byford buick gmc llc,2900 s hwy 81,chickashaok,73018",13710,byford buick gmc llc,1836,2900 s hwy 81,chickasha,ok,73018,(405) 224-0475,NaN,NaN,NaN,NaN,73018
13521,"byford buick gmc llc,2900 u s 81,chickashaok,73018",576069,byford buick gmc llc,206,2900 u s 81,chickasha,ok,73018,(405) 224-0475,www.byfordauto.com,NaN,https://www.facebook.com/byfordbuickgmc/,https://www.cars.com/dealers/184110/byford-buick-gmc/,73018


In [698]:
data_combined.shape

(13523, 14)

### The number of distinct dealers is now down to 13523. which is 51% of actual data. Lets check if we can refine further

In [699]:
data_combined

,meta_data,id,name,tenant_id,address_line,city,state,zip,phone,web,google_url,facebook_url,cars_url,zip_short
0,"001 putnam chevrolet and cadillac,3052 martin luther king junior blvd,burlingameca,94031",1123420,001 putnam chevrolet and cadillac,11138,3052 martin luther king junior blvd,burlingame,ca,94031,(479) 856-6390,www.bigotires.com,NaN,https://www.facebook.com/bigotiresfayetteville/,NaN,94031
1,"002 fremont chevrolet,5850 cushing pkwy,fremontca,94538",1123421,002 fremont chevrolet,11138,5850 cushing pkwy,fremont,ca,94538,(928) 344-2702,NaN,https://maps.google.com/maps?cid=2529089401223706257,https://www.facebook.com/bigotiresyuma004006/,NaN,94538
2,"003 chevrolet of stevens creek,3640 stevens creek blvd,san joseca,95117",1123422,003 chevrolet of stevens creek,11138,3640 stevens creek blvd,san jose,ca,95117,(520) 625-9414,NaN,https://maps.google.com/maps?cid=17294707326087392923,https://www.facebook.com/bigotiresgreenvalley004018/,NaN,95117
3,"004 capitol chevrolet,905 capitol expressway auto mall,san joseca,95136",1123423,004 capitol chevrolet,11138,905 capitol expressway auto mall,san jose,ca,95136,(928) 776-1111,NaN,https://maps.google.com/maps?cid=16621265989335969999,https://www.facebook.com/pages/big-o-tires/142282719150678,NaN,95136
4,"1 brookville chevrolet buick,1 e main st,brookvillepa,15825",18311,1 brookville chevrolet buick,1840,1 e main st,brookville,pa,15825,(814) 849-8313,NaN,NaN,https://www.facebook.com/brookvillechevy/,NaN,15825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13518,"byers kia oh057,5342 columbus pike,lewis centeroh,43035",16039,byers kia oh057,1157,5342 columbus pike,lewis center,oh,43035,NaN,www.byerskia.com,https://maps.google.com/maps?cid=16395390017796853957,NaN,http://www.cars.com/dealers/187255/byers-kia/,43035
13519,"byers mazda,2455 billingsley rd,columbusoh,43235",208328,byers mazda,1057,2455 billingsley rd,columbus,oh,43235 1925,(614) 792-2455,NaN,https://maps.google.com/maps?cid=1129683514267478914,https://www.facebook.com/byersmazdadublin/,https://www.cars.com/dealers/2575096/byers-dublin-mazda/,43235
13520,"byford buick gmc llc,2900 s hwy 81,chickashaok,73018",13710,byford buick gmc llc,1836,2900 s hwy 81,chickasha,ok,73018,(405) 224-0475,NaN,NaN,NaN,NaN,73018
13521,"byford buick gmc llc,2900 u s 81,chickashaok,73018",576069,byford buick gmc llc,206,2900 u s 81,chickasha,ok,73018,(405) 224-0475,www.byfordauto.com,NaN,https://www.facebook.com/byfordbuickgmc/,https://www.cars.com/dealers/184110/byford-buick-gmc/,73018


In [700]:
data_combined.isnull().sum()

meta_data       0   
id              0   
name            0   
tenant_id       0   
address_line    1   
city            0   
state           0   
zip             0   
phone           1141
web             5057
google_url      1711
facebook_url    5634
cars_url        5867
zip_short       0   
dtype: int64

In [701]:
data_combined.count()

meta_data       13523
id              13523
name            13523
tenant_id       13523
address_line    13522
city            13523
state           13523
zip             13523
phone           12382
web             8466 
google_url      11812
facebook_url    7889 
cars_url        7656 
zip_short       13523
dtype: int64

In [702]:
for column in data_combined.columns:
    print("The " + column  + " has following unique elements :" + str(data_combined[column].nunique()) + " out of " + str(data_combined.shape[0]) + " entries")

The meta_data has following unique elements :13523 out of 13523 entries
The id has following unique elements :13523 out of 13523 entries
The name has following unique elements :7970 out of 13523 entries
The tenant_id has following unique elements :481 out of 13523 entries
The address_line has following unique elements :7916 out of 13523 entries
The city has following unique elements :1882 out of 13523 entries
The state has following unique elements :53 out of 13523 entries
The zip has following unique elements :3635 out of 13523 entries
The phone has following unique elements :6287 out of 13523 entries
The web has following unique elements :4450 out of 13523 entries
The google_url has following unique elements :5249 out of 13523 entries
The facebook_url has following unique elements :5651 out of 13523 entries
The cars_url has following unique elements :4388 out of 13523 entries
The zip_short has following unique elements :2750 out of 13523 entries


## Name and address uniquely represent a specific dealer, lets handle overlapping among them.

In [703]:
data_combined['dealer_addresses'] = data_combined['name'] + ',' + data_combined['address_line']

In [704]:
data_combined['dealer_addresses'].value_counts()

audi turnersville,3751 nj 42                                   4
bass mazda,4900 detroit rd                                     4
audi pembroke pines,15000 sheridan st                          4
bell ford,1200 nj 27                                           4
a crivelli buick gmc,1403 allegheny blvd                       4
                                                              ..
bergstrom chevrolet of middleton,3605 tribeca dr               1
autonation chevrolet north richland hills,7769 boulevard 26    1
brotherton cadillac nw,17545 aurora avenue north               1
audi la crosse,726 monitor st                                  1
brown brown chevrolet,145 east main street                     1
Name: dealer_addresses, Length: 11395, dtype: int64

In [705]:
data_combined_v2=data_combined.groupby(['dealer_addresses']).first().reset_index()

In [706]:
data_combined_v2.shape

(11395, 15)

### 3000 entries further reduced.

In [707]:
data_combined_v2.isnull().sum()

dealer_addresses    0   
meta_data           0   
id                  0   
name                0   
tenant_id           0   
address_line        0   
city                0   
state               0   
zip                 0   
phone               1115
web                 3178
google_url          1681
facebook_url        3761
cars_url            4012
zip_short           0   
dtype: int64

In [708]:
for column in data_combined_v2.columns:
    print("The " + column  + " has following unique elements :" + str(data_combined_v2[column].nunique()) + " out of " + str(data_combined_v2.shape[0]) + " entries")

The dealer_addresses has following unique elements :11395 out of 11395 entries
The meta_data has following unique elements :11395 out of 11395 entries
The id has following unique elements :11395 out of 11395 entries
The name has following unique elements :7970 out of 11395 entries
The tenant_id has following unique elements :468 out of 11395 entries
The address_line has following unique elements :7916 out of 11395 entries
The city has following unique elements :1842 out of 11395 entries
The state has following unique elements :53 out of 11395 entries
The zip has following unique elements :3491 out of 11395 entries
The phone has following unique elements :6032 out of 11395 entries
The web has following unique elements :4424 out of 11395 entries
The google_url has following unique elements :5197 out of 11395 entries
The facebook_url has following unique elements :5565 out of 11395 entries
The cars_url has following unique elements :4336 out of 11395 entries
The zip_short has following un

## The uniqueness in phone number is still quite low,

### Single phone number cannot be used by multiple dealers, but there is still scope that a single person is running two dealerships, so having only phone as primary key will not make sense, lets try to combine it with name,addressline and try to club it together to make it a unique entity

In [709]:
data_combined_v2['dealer_contact'] = data_combined_v2['name'].fillna(' ') + ',' + data_combined_v2['address_line'].fillna(' ') +  ',' + data_combined_v2['phone'].fillna(' ')

In [710]:
data_combined_v2['dealer_contact'].nunique()

11395

In [711]:
data_combined_v2['dealer_contact'].value_counts().sort_index(ascending=False)


byford chevrolet buick gmc,1909 u s 81,(580) 255-0550                                   1
byford buick gmc llc,2900 u s 81,(405) 224-0475                                         1
byford buick gmc llc,2900 s hwy 81,(405) 224-0475                                       1
byers mazda,2455 billingsley rd,(614) 792-2455                                          1
byers kia oh057,5342 columbus pike,                                                     1
                                                                                       ..
1 brookville chevrolet buick,1 e main st,(814) 849-8313                                 1
004 capitol chevrolet,905 capitol expressway auto mall,(928) 776-1111                   1
003 chevrolet of stevens creek,3640 stevens creek blvd,(520) 625-9414                   1
002 fremont chevrolet,5850 cushing pkwy,(928) 344-2702                                  1
001 putnam chevrolet and cadillac,3052 martin luther king junior blvd,(479) 856-6390    1
Name: deal

In [712]:
data_combined_v2.shape

(11395, 16)

##### Not much of a result after refining here

## It is clearly visible that still grouping can be done to the data as the unique entries in few columns which are important to identify unique dealer are quite less, for example,combination of website,google and facebook. Lets try to examine that

In [713]:
data_combined_v2.web.value_counts()

www.billionauto.com                       23
www.brucknertruck.com                     13
www.autonationcollisioncenters.com        11
www.autonationcadillacwestamarillo.com    8 
www.honda.com                             8 
                                         .. 
www.acgiant.com                           1 
www.bobmaxeylincoln.com                   1 
www.cochrannissanmonroeville.com          1 
www.bommaritonissanwest.com               1 
www.audimv.com                            1 
Name: web, Length: 4424, dtype: int64

In [635]:
data_combined_v2[data_combined_v2.web=='http://www.billionauto.com/']

,dealer_addresses,meta_data,id,name,tenant_id,address_line,city,state,zip,phone,web,google_url,facebook_url,cars_url,zip_short,dealer_contact


In [714]:
duplicated_internet_presence =  data_combined_v2[data_combined_v2.duplicated(subset = ['web','google_url','facebook_url','cars_url'], keep=False)]

In [715]:
duplicated_internet_presence.shape

(2482, 16)

In [716]:
duplicated_internet_presence

,dealer_addresses,meta_data,id,name,tenant_id,address_line,city,state,zip,phone,web,google_url,facebook_url,cars_url,zip_short,dealer_contact
6,"1 brookville chevrolet buick,30 main st","1 brookville chevrolet buick,30 main st,brookvillepa,15825",575489,1 brookville chevrolet buick,206,30 main st,brookville,pa,15825,(814) 849-8313,www.brookvillegmdealer.com,https://maps.google.com/maps?cid=12622059353105481409,https://www.facebook.com/brookvillechevy/,https://www.cars.com/dealers/154863/1-brookville-chevrolet-buick/,15825,"1 brookville chevrolet buick,30 main st,(814) 849-8313"
7,"1 brookville chevrolet,1 e main st","1 brookville chevrolet,1 e main st,brookvillepa,15825",3244033,1 brookville chevrolet,130469,1 e main st,brookville,pa,15825,(814) 849-8313,www.brookvillegmdealer.com,https://maps.google.com/maps?cid=12622059353105481409,https://www.facebook.com/brookvillechevy/,https://www.cars.com/dealers/154863/1-brookville-chevrolet-buick/,15825,"1 brookville chevrolet,1 e main st,(814) 849-8313"
8,"1 cochran 5,5200 campbells run rd","1 cochran 5,5200 campbells run rd,pittsburghpa,15205",436256,1 cochran 5,8552,5200 campbells run rd,pittsburgh,pa,15205,(877) 290-1514,NaN,NaN,NaN,NaN,15205,"1 cochran 5,5200 campbells run rd,(877) 290-1514"
15,"1 cochran ford of allegheny valley,110 rte 908","1 cochran ford of allegheny valley,110 rte 908,natrona heightsca,15065",1488522,1 cochran ford of allegheny valley,44849,110 rte 908,natrona heights,ca,15065,(877) 844-2719,www.cochranfordnatronaheights.com,https://maps.google.com/maps?cid=13090953824919753129,https://www.facebook.com/body-by-cochran-collision-customization-136810726897323/,https://www.cars.com/dealers/2310238/-1-cochran-chrysler-dodge-jeep-ram/,15065,"1 cochran ford of allegheny valley,110 rte 908,(877) 844-2719"
17,"1 cochran ford,110 rte 908 building 1","1 cochran ford,110 rte 908 building 1,natrona heightspa,15065",3233063,1 cochran ford,130469,110 rte 908 building 1,natrona heights,pa,15065 2810,(412) 245-4320,www.cochranfordnatronaheights.com,https://maps.google.com/maps?cid=13090953824919753129,https://www.facebook.com/body-by-cochran-collision-customization-136810726897323/,https://www.cars.com/dealers/2310238/-1-cochran-chrysler-dodge-jeep-ram/,15065,"1 cochran ford,110 rte 908 building 1,(412) 245-4320"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11360,"burns chevrolet inc,1733 north limestone st","burns chevrolet inc,1733 north limestone st,gaffneysc,29340",45079,burns chevrolet inc,2980,1733 north limestone st,gaffney,sc,29340,(864) 489-6000,NaN,NaN,NaN,NaN,29340,"burns chevrolet inc,1733 north limestone st,(864) 489-6000"
11361,"burnsville chevrolet buick inc,627 w main st","burnsville chevrolet buick inc,627 w main st,burnsvillenc,28714",18220,burnsville chevrolet buick inc,1836,627 w main st,burnsville,nc,28714,(828) 682-6141,NaN,NaN,NaN,NaN,28714,"burnsville chevrolet buick inc,627 w main st,(828) 682-6141"
11363,"burnsville chevrolet buick,627 w main st","burnsville chevrolet buick,627 w main st,burnsvillenc,28714",97092,burnsville chevrolet buick,1840,627 w main st,burnsville,nc,28714,NaN,NaN,NaN,NaN,NaN,28714,"burnsville chevrolet buick,627 w main st,"
11375,"buss automotive inc,1255 e green bay st","buss automotive inc,1255 e green bay st,shawanowi,54166",24884,buss automotive inc,1836,1255 e green bay st,shawano,wi,54166,(715) 524-0444,NaN,NaN,NaN,NaN,54166,"buss automotive inc,1255 e green bay st,(715) 524-0444"


## Clearly visible that few of them belong to same dealer, lets work out on consolidating them.

In [717]:
data_combined_v2['internet_presence'] = data['name'].fillna(' ') + ',' + data['phone'].fillna(' ') + ',' + data['zip_short'].fillna(' ') + ',' + data_combined_v2['web'].fillna(' ') + ',' + data_combined_v2['google_url'].fillna(' ') + ',' + data_combined_v2['facebook_url'].fillna(' ')\
+data_combined_v2['cars_url'].fillna(' ')

In [718]:
data_combined_v2['internet_presence'].value_counts()

auffenberg chevrolet buick gmc,(573) 747-4120,63640, , ,                                                                                                                                                                                                            4
autonation ford westlake 2025, ,44145, , ,                                                                                                                                                                                                                          3
autonation ford westlake,(440) 296-3019,44145, , ,                                                                                                                                                                                                                  3
aber s garage inc,(419) 281-5500,44805, , ,                                                                                                                                                                           

In [719]:
data_combined_v3 =data_combined_v2.groupby(['internet_presence']).first().reset_index()

In [720]:
data_combined_v3.shape

(10842, 17)

In [721]:
data_combined_v3.head(5)

,internet_presence,dealer_addresses,meta_data,id,name,tenant_id,address_line,city,state,zip,phone,web,google_url,facebook_url,cars_url,zip_short,dealer_contact
0,"001 putnam chevrolet and cadillac,(479) 856-6390,94031, ,https://maps.google.com/maps?cid=15640886745739450165,","a sarasota chevrolet new car quotes,bay rd","a sarasota chevrolet new car quotes,bay rd,bay rdfl,34239",1525312,a sarasota chevrolet new car quotes,45332,bay rd,bay rd,fl,34239,(941) 284-6900,NaN,https://maps.google.com/maps?cid=15640886745739450165,NaN,NaN,34239,"a sarasota chevrolet new car quotes,bay rd,(941) 284-6900"
1,"002 fremont chevrolet,(928) 344-2702,94538, ,https://maps.google.com/maps?cid=2859196772727796258,","a sarasota dodge new car quotes,14 holiday dr n","a sarasota dodge new car quotes,14 holiday dr n,sarasotafl,34231",1601117,a sarasota dodge new car quotes,83249,14 holiday dr n,sarasota,fl,34231,(941) 284-6900,NaN,https://maps.google.com/maps?cid=2859196772727796258,NaN,NaN,34231,"a sarasota dodge new car quotes,14 holiday dr n,(941) 284-6900"
2,"003 chevrolet of stevens creek,(520) 625-9414,95117, ,https://maps.google.com/maps?cid=14703233780577702587,","a sarasota honda new car quotes,caribbean dr","a sarasota honda new car quotes,caribbean dr,sarasotafl,34231",1482254,a sarasota honda new car quotes,44849,caribbean dr,sarasota,fl,34231,(941) 284-6900,NaN,https://maps.google.com/maps?cid=14703233780577702587,NaN,NaN,34231,"a sarasota honda new car quotes,caribbean dr,(941) 284-6900"
3,"004 capitol chevrolet,(928) 776-1111,95136, ,https://maps.google.com/maps?cid=14703233780577702587,","a sarasota honda new car quotes,mileground rd","a sarasota honda new car quotes,mileground rd,mileground rdwv,26505",1525184,a sarasota honda new car quotes,45332,mileground rd,mileground rd,wv,26505,NaN,NaN,https://maps.google.com/maps?cid=14703233780577702587,NaN,NaN,26505,"a sarasota honda new car quotes,mileground rd,"
4,"1 brookville chevrolet buick,(814) 246-2055,15825, ,https://maps.google.com/maps?cid=12622059353105481409,https://www.facebook.com/brookvillechevy/http://www.cars.com/dealers/154863/1-brookville-chevrolet-buick/","1 brookville chevrolet buick,1 east main st","1 brookville chevrolet buick,1 east main st,brookvillepa,15825",118193,1 brookville chevrolet buick,2980,1 east main st,brookville,pa,15825,(814) 849-8313,NaN,https://maps.google.com/maps?cid=12622059353105481409,https://www.facebook.com/brookvillechevy/,http://www.cars.com/dealers/154863/1-brookville-chevrolet-buick/,15825,"1 brookville chevrolet buick,1 east main st,(814) 849-8313"


In [722]:
data_combined_count = data_combined_v3.groupby(['web','google_url','facebook_url','cars_url'])['web','google_url','facebook_url','cars_url'].count()
data_combined_count

,,,,web,google_url,facebook_url,cars_url
web,google_url,facebook_url,cars_url,,,,
171nissan.com,https://maps.google.com/maps?cid=3971356509180237363,https://www.facebook.com/171nissan/,https://www.cars.com/dealers/211161/171-nissan/,1,1,1,1
academyforddealer.com,https://maps.google.com/maps?cid=11970897632866746619,https://www.facebook.com/academyfordsales,https://www.cars.com/dealers/2208/academy-ford/,1,1,1,1
acrivellisubaru.com,https://maps.google.com/maps?cid=914417773313057702,https://www.facebook.com/acrivelliautogroup/,https://www.cars.com/dealers/155193/a-crivelli-chevrolet/,1,1,1,1
acuraofbayshore.com,https://maps.google.com/maps?cid=40693424694342408,https://www.facebook.com/bayshoreacura/,https://www.cars.com/dealers/5729/acura-of-bay-shore/,1,1,1,1
acuraofberlin.com,https://maps.google.com/maps?cid=5883875942797871769,https://www.facebook.com/acuraberlin/,https://www.cars.com/dealers/4101/acura-of-berlin/,1,1,1,1
...,...,...,...,...,...,...,...
www.wheelingchevy.com,https://maps.google.com/maps?cid=2923692249652154434,https://www.facebook.com/billstasekchevrolet/,http://www.cars.com/dealers/602/bill-stasek-chevrolet/,1,1,1,1
www.whiteallenaudiofdayton.com,https://maps.google.com/maps?cid=279212125703957825,https://www.facebook.com/whitealleneuro/,https://www.cars.com/dealers/14697/white-allen-european-auto-group/,1,1,1,1
www.whitemotor.com,https://maps.google.com/maps?cid=10509705568694096173,https://www.facebook.com/bfwhitemotorco/,http://www.cars.com/dealers/148335/bf-white-motor-company-of-stanton-inc/,1,1,1,1


In [723]:
data_combined_v3.name.value_counts()

anderson ford                 12
bruckner s mack volvo         10
all american chevrolet        9 
american honda motor co       8 
affordable autos              7 
                             .. 
basin mitsubishi              1 
accessories of florida        1 
all star pontiac gmc truck    1 
avis ford inc legacy          1 
bmw of toledo                 1 
Name: name, Length: 7764, dtype: int64

## Examining why one name is repeated 12 times

In [724]:
data_combined_v3[data_combined_v3.name == 'anderson ford']

,internet_presence,dealer_addresses,meta_data,id,name,tenant_id,address_line,city,state,zip,phone,web,google_url,facebook_url,cars_url,zip_short,dealer_contact
980,"acura of glendale,(818) 502-1100,91204, ,https://maps.google.com/maps?cid=8999307113185777568,https://www.facebook.com/andersonfordclintonilhttps://www.cars.com/dealers/153453/anderson-ford/","anderson ford,1001 state route 10 w","anderson ford,1001 state route 10 w,clintonil,61727",2825439,anderson ford,119451,1001 state route 10 w,clinton,il,61727,(877) 883-1717,NaN,https://maps.google.com/maps?cid=8999307113185777568,https://www.facebook.com/andersonfordclintonil,https://www.cars.com/dealers/153453/anderson-ford/,61727,"anderson ford,1001 state route 10 w,(877) 883-1717"
981,"acura of glendale,(818) 502-1100,91204,www.anderson-ford.net,https://maps.google.com/maps?cid=8999307113185777568,","anderson ford,1001 illinois 10","anderson ford,1001 illinois 10,clintonil,61727",129655,anderson ford,4046,1001 illinois 10,clinton,il,61727,(217) 935-3106,www.anderson-ford.net,https://maps.google.com/maps?cid=8999307113185777568,NaN,NaN,61727,"anderson ford,1001 illinois 10,(217) 935-3106"
983,"acura of glendale,(818) 502-1100,91204,www.andersonfordgibsoncity.com,https://maps.google.com/maps?cid=9078969412338938237,https://www.facebook.com/pages/anderson-ford/126323237423314https://www.cars.com/dealers/199492/anderson-ford/","anderson ford,124 s sangamon ave","anderson ford,124 s sangamon ave,gibson cityca,60936",1487454,anderson ford,44849,124 s sangamon ave,gibson city,ca,60936,(217) 784-4119,www.andersonfordgibsoncity.com,https://maps.google.com/maps?cid=9078969412338938237,https://www.facebook.com/pages/anderson-ford/126323237423314,https://www.cars.com/dealers/199492/anderson-ford/,60936,"anderson ford,124 s sangamon ave,(217) 784-4119"
984,"acura of glendale,(888) 670-9904,91204,www.andersonforddouglas.com,https://maps.google.com/maps?cid=1772629934778698630,https://www.facebook.com/andersonforddouglas/http://www.cars.com/dealers/148152/prince-ford-inc/","anderson ford,109 westgreen rd","anderson ford,109 westgreen rd,douglasga,31533",2897231,anderson ford,289,109 westgreen rd,douglas,ga,31533,(912) 384-2600,www.andersonforddouglas.com,https://maps.google.com/maps?cid=1772629934778698630,https://www.facebook.com/andersonforddouglas/,http://www.cars.com/dealers/148152/prince-ford-inc/,31533,"anderson ford,109 westgreen rd,(912) 384-2600"
985,"acura of honolulu,(808) 829-3073,96819, ,https://maps.google.com/maps?cid=9078969412338938237,","anderson ford,124 south sangamon ave","anderson ford,124 south sangamon ave,gibson cityil,60936",159681,anderson ford,4046,124 south sangamon ave,gibson city,il,60936,(217) 784-4119,NaN,https://maps.google.com/maps?cid=9078969412338938237,NaN,NaN,60936,"anderson ford,124 south sangamon ave,(217) 784-4119"
986,"acura of honolulu,(808) 829-3073,96819,www.andersonford.info,https://maps.google.com/maps?cid=18316833063836885106,https://www.facebook.com/anderson-ford-529444370443925https://www.cars.com/dealers/147827/anderson-ford/","anderson ford,13872 us 59","anderson ford,13872 us 59,clevelandmt,77327",1491324,anderson ford,44849,13872 us 59,cleveland,mt,77327,(888) 871-9716,www.andersonford.info,https://maps.google.com/maps?cid=18316833063836885106,https://www.facebook.com/anderson-ford-529444370443925,https://www.cars.com/dealers/147827/anderson-ford/,77327,"anderson ford,13872 us 59,(888) 871-9716"
987,"acura of honolulu,(808) 829-3073,96819,www.andersonford.info,https://maps.google.com/maps?cid=5290594378440192604,https://www.facebook.com/anderson-ford-529444370443925","anderson ford,13872 us 59 s","anderson ford,13872 us 59 s,clevelandtx,77328",3238506,anderson ford,130469,13872 us 59 s,cleveland,tx,77328,(281) 545-7162,www.andersonford.info,https://maps.google.com/maps?cid=5290594378440192604,https://www.facebook.com/anderson-ford-529444370443925,NaN,77328,"anderson ford,13872 us 59 s,(281) 545-7162"
988,"acura of honolulu,(808) 942

## There are still mentions of same dealer multiple times, but a combinaton of name, tenant_id and zip might help in reducing the dealer list further, consolidating them

In [725]:
data_combined_v4 =data_combined_v3.groupby(['name','city','zip_short']).first().reset_index()

In [726]:
data_combined_v4.shape

(8342, 17)

### Data further reduced to 8342

In [727]:
data_combined_v4.isnull().sum()

name                 0   
city                 0   
zip_short            0   
internet_presence    0   
dealer_addresses     0   
meta_data            0   
id                   0   
tenant_id            0   
address_line         0   
state                0   
zip                  0   
phone                900 
web                  2038
google_url           1127
facebook_url         2723
cars_url             2986
dealer_contact       0   
dtype: int64

In [728]:
data_combined_v4[data_combined_v4.name == 'anderson ford']

,name,city,zip_short,internet_presence,dealer_addresses,meta_data,id,tenant_id,address_line,state,zip,phone,web,google_url,facebook_url,cars_url,dealer_contact
1369,anderson ford,anderson,29621,"acura of honolulu,(808) 942-4555,96813, ,https://maps.google.com/maps?cid=1198857466292530781,https://www.facebook.com/andersonfordsc","anderson ford,3900 clemson boulevard","anderson ford,3900 clemson boulevard,andersonsc,29621",2821208,119401,3900 clemson boulevard,sc,29621,(864) 810-1036,www.andersonfordofsc.com,https://maps.google.com/maps?cid=1198857466292530781,https://www.facebook.com/andersonfordsc,https://www.cars.com/dealers/149680/anderson-ford/,"anderson ford,3900 clemson boulevard,(864) 810-1036"
1370,anderson ford,cleveland,77327,"acura of honolulu,(808) 829-3073,96819,www.andersonford.info,https://maps.google.com/maps?cid=18316833063836885106,https://www.facebook.com/anderson-ford-529444370443925https://www.cars.com/dealers/147827/anderson-ford/","anderson ford,13872 us 59","anderson ford,13872 us 59,clevelandmt,77327",1491324,44849,13872 us 59,mt,77327,(888) 871-9716,www.andersonford.info,https://maps.google.com/maps?cid=18316833063836885106,https://www.facebook.com/anderson-ford-529444370443925,https://www.cars.com/dealers/147827/anderson-ford/,"anderson ford,13872 us 59,(888) 871-9716"
1371,anderson ford,cleveland,77328,"acura of honolulu,(808) 829-3073,96819,www.andersonford.info,https://maps.google.com/maps?cid=5290594378440192604,https://www.facebook.com/anderson-ford-529444370443925","anderson ford,13872 us 59 s","anderson ford,13872 us 59 s,clevelandtx,77328",3238506,130469,13872 us 59 s,tx,77328,(281) 545-7162,www.andersonford.info,https://maps.google.com/maps?cid=5290594378440192604,https://www.facebook.com/anderson-ford-529444370443925,https://www.cars.com/dealers/147827/demontrond-ford/,"anderson ford,13872 us 59 s,(281) 545-7162"
1372,anderson ford,clinton,61727,"acura of glendale,(818) 502-1100,91204, ,https://maps.google.com/maps?cid=8999307113185777568,https://www.facebook.com/andersonfordclintonilhttps://www.cars.com/dealers/153453/anderson-ford/","anderson ford,1001 state route 10 w","anderson ford,1001 state route 10 w,clintonil,61727",2825439,119451,1001 state route 10 w,il,61727,(877) 883-1717,www.anderson-ford.net,https://maps.google.com/maps?cid=8999307113185777568,https://www.facebook.com/andersonfordclintonil,https://www.cars.com/dealers/153453/anderson-ford/,"anderson ford,1001 state route 10 w,(877) 883-1717"
1373,anderson ford,douglas,31533,"acura of glendale,(888) 670-9904,91204,www.andersonforddouglas.com,https://maps.google.com/maps?cid=1772629934778698630,https://www.facebook.com/andersonforddouglas/http://www.cars.com/dealers/148152/prince-ford-inc/","anderson ford,109 westgreen rd","anderson ford,109 westgreen rd,douglasga,31533",2897231,289,109 westgreen rd,ga,31533,(912) 384-2600,www.andersonforddouglas.com,https://maps.google.com/maps?cid=1772629934778698630,https://www.facebook.com/andersonforddouglas/,http://www.cars.com/dealers/148152/prince-ford-inc/,"anderson ford,109 westgreen rd,(912) 384-2600"
1374,anderson ford,gibson city,60936,"acura of glendale,(818) 502-1100,91204,www.andersonfordgibsoncity.com,https://maps.google.com/maps?cid=9078969412338938237,https://www.facebook.com/pages/anderson-ford/126323237423314https://www.cars.com/dealers/199492/anderson-ford/","anderson ford,124 s sangamon ave","anderson ford,124 s sangamon ave,gibson cityca,60936",1487454,44849,124 s sangamon ave,ca,60936,(217) 784-4119,www.andersonfordgibsoncity.com,https://maps.google.com/maps?cid=9078969412338938237,https://www.facebook.com/pages/anderson-ford/126323237423314,https://www.cars.com/dealers/199492/anderson-ford/,"anderson ford,124 s sangamon ave,(217) 784-4119"


In [729]:
data_combined_v4.name.value_counts()

bruckner s mack volvo                  10
american honda motor co                8 
affordable autos                       7 
affordable auto sales                  7 
auto smart                             6 
                                      .. 
bergstrom mercedes benz of appleton    1 
audi stevens creek                     1 
bmw of santa maria                     1 
bison motor company                    1 
audi las vegas                         1 
Name: name, Length: 7764, dtype: int64

In [730]:
data_combined_v4[data_combined_v4.name == 'bruckner s mack volvo']

,name,city,zip_short,internet_presence,dealer_addresses,meta_data,id,tenant_id,address_line,state,zip,phone,web,google_url,facebook_url,cars_url,dealer_contact
8089,bruckner s mack volvo,albuquerque,87121,"autonation ford lincoln wolfchase 2363, ,38133,www.brucknertruck.com,https://maps.google.com/maps?cid=7718685067242739490,","bruckner s mack volvo,8101 daytona rd nw","bruckner s mack volvo,8101 daytona rd nw,albuquerquenm,87121",1594486,57705,8101 daytona rd nw,nm,87121,(877) 999-2451,www.brucknertruck.com,https://maps.google.com/maps?cid=7718685067242739490,NaN,NaN,"bruckner s mack volvo,8101 daytona rd nw,(877) 999-2451"
8090,bruckner s mack volvo,amarillo,79118,"autonation ford lincoln wolfchase 2363, ,38133,www.brucknertruck.com,https://maps.google.com/maps?cid=5404558478043464866,https://www.facebook.com/brucknertrucksales/","bruckner s mack volvo,9471 i 40 east exit 76","bruckner s mack volvo,9471 i 40 east exit 76,amarillotx,79118",1518181,45332,9471 i 40 east exit 76,tx,79118,(877) 999-8071,www.brucknertruck.com,https://maps.google.com/maps?cid=5404558478043464866,https://www.facebook.com/brucknertrucksales/,NaN,"bruckner s mack volvo,9471 i 40 east exit 76,(877) 999-8071"
8091,bruckner s mack volvo,bossier city,71111,"autonation ford lincoln wolfchase 2363, ,38133,www.brucknertruck.com,https://maps.google.com/maps?cid=11146096563401691841,https://www.facebook.com/pages/bruckner-truck-sales-inc/245550159595462","bruckner s mack volvo,4275 meadow ln","bruckner s mack volvo,4275 meadow ln,bossier cityla,71111",1507833,45332,4275 meadow ln,la,71111,(866) 928-6580,www.brucknertruck.com,https://maps.google.com/maps?cid=11146096563401691841,https://www.facebook.com/pages/bruckner-truck-sales-inc/245550159595462,NaN,"bruckner s mack volvo,4275 meadow ln,(866) 928-6580"
8092,bruckner s mack volvo,farmington,87401,"autonation ford lincoln wolfchase 2363, ,38133,www.brucknertruck.com,https://maps.google.com/maps?cid=9380787747266475927,https://www.facebook.com/brucknertruckfarmingtonnm/","bruckner s mack volvo,4050 bloomfield hwy","bruckner s mack volvo,4050 bloomfield hwy,farmingtonnm,87401",2739739,114950,4050 bloomfield hwy,nm,87401,(877) 999-2971,www.brucknertruck.com,https://maps.google.com/maps?cid=9380787747266475927,https://www.facebook.com/brucknertruckfarmingtonnm/,NaN,"bruckner s mack volvo,4050 bloomfield hwy,(877) 999-2971"
8093,bruckner s mack volvo,garden city,67846,"autonation ford lincoln wolfchase,(901) 209-1257,38133,www.brucknertruck.com,https://maps.google.com/maps?cid=10582764957551610916,","bruckner s mack volvo,2509 n taylor ave","bruckner s mack volvo,2509 n taylor ave,garden cityks,67846",2743175,115099,2509 n taylor ave,ks,67846,(888) 999-7810,www.brucknertruck.com,https://maps.google.com/maps?cid=10582764957551610916,NaN,NaN,"bruckner s mack volvo,2509 n taylor ave,(888) 999-7810"
8094,bruckner s mack volvo,hays,67601,"autonation ford lincoln union city 2325, ,30291,www.brucknertruck.com,https://maps.google.com/maps?cid=8959750749014169057,https://www.facebook.com/brucknerhays/","bruckner s mack volvo,2101 commerce pkwy","bruckner s mack volvo,2101 commerce pkwy,haysks,67601",1513651,45332,2101 commerce pkwy,ks,67601,(855) 615-1343,www.brucknertruck.com,https://maps.google.com/maps?cid=8959750749014169057,https://www.facebook.com/brucknerhays/,NaN,"bruckner s mack volvo,2101 commerce pkwy,(855) 615-1343"
8095,bruckner s mack volvo,hobbs,88240,"autonation ford lincoln union city 2325, ,30291,www.brucknertruck.com,https://maps.google.com/maps?cid=11187758810578127375,https://www.facebook.com/pages/bruckners-truck-sales/167522453260378","bruckner s mack volvo,2121 nw county rd","bruckner s mack volvo,2121 nw county rd,hobbsnm,88240",1512170,45332,2121 nw county rd,nm,88240,(877) 999-1440,www.brucknertruck.com,https://maps.google.com/maps?cid=11187758810578127375,https://www.facebook.com/pages/bruckners-truck-sales/167522453260378,NaN,"bruckner s mack volvo,2121 nw county rd,(877) 999-1440"
8096,bruckn

In [731]:
data_combined_v4.shape

(8342, 17)

### Lets try to consolidate based on web address to further refine

In [733]:
data_combined_v4.web.head(10)

0    www.bigotires.com           
1    NaN                         
2    NaN                         
3    NaN                         
4    www.brookvillegmdealer.com  
5    www.brookvillegmdealer.com  
6    NaN                         
7    NaN                         
8    www.cochranofmonroeville.com
9    www.cochranofrobinson.com   
Name: web, dtype: object

In [734]:
data_combined_v4.web.value_counts()

www.billionauto.com                        20
www.brucknertruck.com                      11
www.autonationcollisioncenters.com         11
www.autonation.com                         7 
www.honda.com                              6 
                                          .. 
www.actionautochryslerdodgejeep.com        1 
www.bobmooreaudiok.com                     1 
www.billybenderchryslerjeepdodgeram.com    1 
www.billingsleyflm.com                     1 
www.audimv.com                             1 
Name: web, Length: 4215, dtype: int64

In [735]:
data_combined_v4[data_combined_v3.web == 'www.billionauto.com']

,name,city,zip_short,internet_presence,dealer_addresses,meta_data,id,tenant_id,address_line,state,zip,phone,web,google_url,facebook_url,cars_url,dealer_contact
7263,bosak chrysler,merrillville,46410,"autonation acura spokane valley service center,(509) 289-5612,99212, ,https://maps.google.com/?cid=9588476791745658, http://www.cars.com/dealers/155307/bosak-chrysler-jeep-dodge-ram/","bosak chrysler,3111 east lincoln hwy","bosak chrysler,3111 east lincoln hwy,merrillvillein,46410",194774,4634,3111 east lincoln hwy,in,46410,(219) 738-2323,NaN,https://maps.google.com/?cid=9588476791745658,NaN,http://www.cars.com/dealers/155307/bosak-chrysler-jeep-dodge-ram/,"bosak chrysler,3111 east lincoln hwy,(219) 738-2323"
7264,bosak ford,chesterton,46304,"autonation acura stevens creek,(408) 758-5502,95051,www.lakeshoreford.net,https://maps.google.com/maps?cid=10056049863143110313,https://www.facebook.com/179118028045https://www.cars.com/dealers/14973/lake-shore-ford/","bosak ford,244 melton road","bosak ford,244 melton road,chestertonin,46304",3327701,289,244 melton road,in,46304 9434,(219) 787-8600,www.lakeshoreford.net,https://maps.google.com/maps?cid=10056049863143110313,https://www.facebook.com/179118028045,https://www.cars.com/dealers/14973/lake-shore-ford/,"bosak ford,244 melton road,(219) 787-8600"
7265,bosak honda,highland,46322,"autonation acura stevens creek service center,(408) 780-1652,95051,www.bosakhondahighland.com,https://maps.google.com/maps?cid=16265415469065877319,https://www.facebook.com/167889936602685https://www.cars.com/dealers/82/bosak-honda/","bosak honda,9800 indianapolis blvd","bosak honda,9800 indianapolis blvd,highlandin,46322",3004654,123467,9800 indianapolis blvd,in,46322,(219) 922-3100,www.bosakhondahighland.com,https://maps.google.com/maps?cid=16265415469065877319,https://www.facebook.com/167889936602685,https://www.cars.com/dealers/82/bosak-honda/,"bosak honda,9800 indianapolis blvd,(219) 922-3100"
7266,bosak honda,michigan city,46360,"autonation acura stevens creek 2674, ,95051,www.bosakhondamc.com,https://maps.google.com/maps?cid=7865471118760769803,https://www.facebook.com/314891999293https://www.cars.com/dealers/15007/bosak-honda-in-michigan-city/","bosak honda,700 u s 20","bosak honda,700 u s 20,michigan cityin,46360",3004959,123467,700 u s 20,in,46360,(219) 874-4293,www.bosakhondamc.com,https://maps.google.com/maps?cid=7865471118760769803,https://www.facebook.com/314891999293,https://www.cars.com/dealers/15007/bosak-honda-in-michigan-city/,"bosak honda,700 u s 20,(219) 874-4293"
7267,bosak honda highland,highland,46322,"autonation acura stevens creek,(408) 758-5502,95051,www.bosakhondahighland.com,https://maps.google.com/maps?cid=16265415469065877319,https://www.facebook.com/bosakhondahighland/https://www.cars.com/dealers/82/bosak-honda/","bosak honda highland,9800 indianapolis blvd","bosak honda highland,9800 indianapolis blvd,highlandco,46322",1488672,44849,9800 indianapolis blvd,co,46322,(219) 922-3100,www.bosakhondahighland.com,https://maps.google.com/maps?cid=16265415469065877319,https://www.facebook.com/bosakhondahighland/,https://www.cars.com/dealers/82/bosak-honda/,"bosak honda highland,9800 indianapolis blvd,(219) 922-3100"
7273,bosak nissan,burns harbor,46304,"autonation acura of stevens creek,(408) 758-5502,95051,www.bosaknissan.com,https://maps.google.com/maps?cid=14062767460558294530,https://www.facebook.com/chestertonnissan/https://www.cars.com/dealers/198477/bob-rohrman-nissan-of-chesterton-in-burns-harbor/","bosak nissan,220 verplank rd","bosak nissan,220 verplank rd,burns harborin,46304",2975957,120898,220 verplank rd,in,46304,(219) 787-1777,www.bosaknissan.com,https://maps.google.com/maps?cid=14062767460558294530,https://www.facebook.com/chestertonnissan/,https://www.cars.com/dealers/198477/bob-rohrman-nissan-of-chesterton-in-burns-harbor/,"bosak nissan,220 verplank rd,(219) 787-1777"
7274,boshears ford,marshall,49068,"autonation alfa romeo north denver parts center,(844) 427-4422,8023

## No notable same references found on manual vision but still...

###  Slowly it can be refined further with more time..  with given time constraints, I stop here. Now lets build a system to map the canonical list to new entries.



# Lets assign an ID for the canonical list

In [507]:
data_combined_v4['canonical_id'] = data_combined_v3.index + 1

In [509]:
data_combined_v3.columns

Index(['name', 'city', 'zip_short', 'dealer_addresses', 'meta_data', 'id',
       'tenant_id', 'address_line', 'state', 'zip', 'phone', 'web',
       'google_url', 'facebook_url', 'cars_url', 'dealer_contact',
       'internet_presence', 'canonical_id'],
      dtype='object')

In [512]:
canonical_df =data_combined_v3[['canonical_id','name','city','zip_short','address_line','state','zip','phone','web','google_url','facebook_url','cars_url']]

##  The process has following methods.

##### Clean and refine data.
##### compare it with canonical list.
##### Get top 1 value with its confidence.
#### If confidence is well below certain level, we should update and add it to canonical list.

# The entire functional logic is written in datamapper.py



## Building a model for comparision can be done through certain similarty techniques like : 

1. Levenstein distance
2. Cosine Similarity
3. Fuzzy search etc.


Lets build confidence based on similarity score.

Implementing FuzzyWuzzy for a test data as below

In [736]:
canonical_df.columns

Index(['canonical_id', 'name', 'city', 'zip_short', 'address_line', 'state',
       'zip', 'phone', 'web', 'google_url', 'facebook_url', 'cars_url',
       'meta_data'],
      dtype='object')

In [513]:
canonical_df['meta_data'] = canonical_df['name'].fillna(' ') + ',' + canonical_df['address_line'].fillna(' ') + ',' + canonical_df['city'].fillna(' ')\
+canonical_df['state'].fillna(' ') + ',' + canonical_df['zip_short'].fillna(' ') + ',' + canonical_df['web'].fillna(' ') + ',' + canonical_df['phone'].fillna(' ')

C:\Users\ynagubandi\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [514]:
canonical_df['meta_data'].head(5)

0    001 putnam chevrolet and cadillac,3052 martin ...
1    002 fremont chevrolet,5850 cushing pkwy,fremon...
2    003 chevrolet of stevens creek,3640 stevens cr...
3    004 capitol chevrolet,905 capitol expressway a...
4    1 brookville chevrolet,1 e main st,brookvillep...
Name: meta_data, dtype: object

In [518]:
test = data['meta_data'].iloc[1]
test

'1 brookville chevrolet buick,1 e main st,brookvillepa,15825'

In [532]:
fuzz_list = canonical_df['meta_data'].apply(lambda x: fuzz.ratio(x,test))

In [560]:
max(fuzz_list)

81

In [531]:
fuzz_list = max(fuzz_list)

In [562]:
print(canonical_df[canonical_df.canonical_id == (fuzz_list[fuzz_list == max(fuzz_list)].index[0]+1)][['meta_data','canonical_id']])

                                                                                meta_data  \
5  1 brookville chevrolet buick,1 e main st,brookvillepa,15825,www.brookvillegmdealer.com   

   canonical_id  
5  6             


In [558]:
test

'1 brookville chevrolet buick,1 e main st,brookvillepa,15825'

In [740]:
process.extract(test,canonical_df.meta_data,scorer=fuzz.token_sort_ratio)

In [745]:
test_answer

('1 brookville chevrolet buick,1 e main st,brookvillepa,15825,www.brookvillegmdealer.com',
 81,
 5)

## Both are reasonablly similar, so started implementing Data mapper at this point.

# Test the class and functions created Datamapper.py .


In [17]:
import datamapper

In [18]:
sample = data[0:1000]

In [12]:
canonical_list = datamapper.prepare_canonical_dataframe(sample)

Begin data cleaning
The id has following unique elements :979 out of 30000 entries
The name has following unique elements :321 out of 30000 entries
The tenant_id has following unique elements :87 out of 30000 entries
The address_line has following unique elements :377 out of 30000 entries
The city has following unique elements :210 out of 30000 entries
The state has following unique elements :44 out of 30000 entries
The zip has following unique elements :260 out of 30000 entries
The phone has following unique elements :291 out of 30000 entries
The web has following unique elements :263 out of 30000 entries
The google_url has following unique elements :271 out of 30000 entries
The facebook_url has following unique elements :300 out of 30000 entries
The cars_url has following unique elements :273 out of 30000 entries
data cleaned
data reduced to 320


# For first 1000 entries canonical list has been reduced to 320.

Now lets map them back with the 100 entries

In [19]:
map_raw_data = datamapper.map_data_to_canonical_data(sample, canonical_list)

Begin data cleaning
The id has following unique elements :979 out of 30000 entries
The name has following unique elements :321 out of 30000 entries
The tenant_id has following unique elements :87 out of 30000 entries
The address_line has following unique elements :377 out of 30000 entries
The city has following unique elements :210 out of 30000 entries
The state has following unique elements :44 out of 30000 entries
The zip has following unique elements :260 out of 30000 entries
The phone has following unique elements :291 out of 30000 entries
The web has following unique elements :263 out of 30000 entries
The google_url has following unique elements :271 out of 30000 entries
The facebook_url has following unique elements :300 out of 30000 entries
The cars_url has following unique elements :273 out of 30000 entries
data cleaned
(1000, 2)


In [22]:
map_raw_data[['meta_data','mapping_canonical_id','confidence']]

,meta_data,mapping_canonical_id,confidence
0,"1 brookville chevrolet,1 e main st,brookvillepa,15825,www.brookvillegmdealer.com,(814) 849-8313",5,100
1,"1 brookville chevrolet buick,1 e main st,brookvillepa,15825, ,(814) 849-8313",6,84
2,"1 brookville chevrolet buick,1 east main st,brookvillepa,15825, ,(814) 849-8313",6,83
3,"1 brookville chevrolet buick,1 e main st,brookvillepa,15825, ,(814) 849-8313",6,84
4,"1 brookville chevrolet buick,30 main st,brookvillepa,15825,www.brookvillegmdealer.com,(814) 849-8313",6,96
...,...,...,...
995,"armen chevrolet saab of ardmore,125 lancaster ave,ardmorepa,19003, ,(610) 649-0300",318,96
996,"arnell chevrolet inc,239 melton rd,chestertonin,46304, ,(219) 787-9300",319,82
997,"arnell chevrolet inc,239 melton rd,chestertonin,46304, ,(219) 787-9300",319,82
998,"arnell chevrolet inc,239 melton rd,burns harborin,46304, ,(219) 787-9300",319,100


In [25]:
map_raw_data['data_quality'] = 100-(map_raw_data.isnull().sum(axis=1)/len(map_raw_data.columns))*100

In [26]:
map_raw_data['data_quality']

0      100.00
1      81.25 
2      93.75 
3      75.00 
4      100.00
        ...  
995    93.75 
996    75.00 
997    81.25 
998    81.25 
999    100.00
Name: data_quality, Length: 1000, dtype: float64

# Conclusion :

1. Data cleaned and refined

2. Best estimate of cannical list extracted from data.

3. All other metrics like web address,google_urls and websites are well preserved wherever available through data consolidation.

4. System created with a new primary key canonical_id and a model based on similarty score for creating mappings 

5. A new script datamapper.py is created to map the raw_data based on canonical list generated, which can be joined using canonical_id in list and mapping_canonical_id in raw_data.

6. Data quality computed for raw_data

7. Provision to update canonical list is also provided to scale the service when in production.

8. 3 suggestive deployment ways are created (Microservice approach, Heroku based Deployment, Lambda in AWS)

9. Further scope and direction provided in word document.

10. Operational instructions for datamapper and how it can be further improved provided in walkthrough document



# Playground (for rough work)

In [496]:
import re

In [497]:
sentance = "hello there 1 is here for you"
re.sub(r'(?<!\S)(\d)(?!\S)' , r'_suff.\2',sentance)

error: invalid group reference 2 at position 7

In [ ]:
# ROUGH

# pattern = r'\b({})\b'.format('|'.join(sorted(re.escape(number) for number in number_map)))
# address2 = "one ,123 north anywhere southstreet"
# re.sub(pattern, lambda m: number_map.get(m.group(0).lower()), address2, flags=re.IGNORECASE)


In [ ]:
#df['date'].replace('^([0-9])$', number_map'\1', regex=True)
#data['meta_data'] = data['meta_data'].replace('(\s|,)?(\d)(\s|,)')
#data['meta_data'].astype(str).apply(lambda row: re.sub(r'(?<!\S)\d+(?!\S)', lambda x: p.number_to_words(x.group()), row))